In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#install

In [2]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!pip install bert-sklearn/.

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 1.25 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Processing ./bert-sklearn
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 131 kB 13.2 MB/s 
     |████████████████████████████████| 8.4 MB 37.0 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 138 kB 53.6 MB/s 
     |████████████████████████████████| 127 kB 54.2 MB/s 
  Created wheel for bert-sklearn: filename=bert_

In [3]:
import os
import math
import random
import csv
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import statistics as stats

from bert_sklearn import BertClassifier
from bert_sklearn import BertRegressor
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model

# **Import and load data**

In [4]:
ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/data mining/'
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/data mining/clear_text.xlsx'
TEST_PATH = '/content/drive/MyDrive/Colab Notebooks/data mining/clear_testtext.xlsx'

In [5]:
df = pd.read_excel(DATA_PATH)
df.tail()

,true_label,text,len,label
31517,__label__Do_an_va_do_uong,món kim chi ngon mùa thu ực ực ực chảy nước_mi...,40,3
31518,__label__Nha_dat,thuê chung_cư phạm văn đồng bắc từ_liêm hà_nội...,119,15
31519,__label__Kinh_doanh_va_Cong_nghiep,trình học_phí tặng khoá học kế_toán thuế trị_g...,91,9
31520,__label__Nha_va_vuon,bố_trí thông_minh giúp nhà_ống sài_gòn chỗ tối...,181,16
31521,__label__Kinh_doanh_va_Cong_nghiep,chúc_mừng sinh_nhật hoàng_gia media group tròn...,80,9


# **Process data**

In [6]:
X_train, Y_train = df['text'], df['label']

# **BERT model**

In [7]:
len_seq = 158
model = BertClassifier(max_seq_length=len_seq,
                       train_batch_size=16,
                       epochs=6,
                       bert_model='bert-base-multilingual-cased', validation_fraction=0.)
model

Building sklearn text classifier...


BertClassifier(bert_model='bert-base-multilingual-cased', epochs=6,
               max_seq_length=158, train_batch_size=16,
               validation_fraction=0.0)

## Train

In [8]:
history = model.fit(X_train, Y_train)

Training  :   0%|          | 0/1971 [00:00<?, ?it/s]

Training  :   0%|          | 0/1971 [00:00<?, ?it/s]

Training  :   0%|          | 0/1971 [00:00<?, ?it/s]

Training  :   0%|          | 0/1971 [00:00<?, ?it/s]

## evaluate

In [ ]:
# make predictions
y_pred = model.predict(X_test)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder().fit(df.true_label)

In [ ]:
true_labels = encoder.inverse_transform(range(23))

In [ ]:
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred, Y_test) * 100))
print(classification_report(Y_test, y_pred, target_names=true_labels))

Accuracy: 93.02%
                                       precision    recall  f1-score   support

                   __label__Chinh_tri       1.00      1.00      1.00       294
         __label__Con_nguoi_va_xa_hoi       0.99      1.00      1.00       160
               __label__Cong_nghe_moi       1.00      1.00      1.00        14
            __label__Do_an_va_do_uong       1.00      1.00      1.00       961
                     __label__Du_lich       1.00      1.00      1.00       367
                    __label__Giai_tri       1.00      1.00      1.00        87
                    __label__Giao_duc       1.00      1.00      1.00       256
                  __label__Giao_thong       1.00      1.00      1.00        37
                    __label__Khoa_hoc       1.00      1.00      1.00        59
   __label__Kinh_doanh_va_Cong_nghiep       0.80      0.72      0.75       934
         __label__Lam_dep_va_the_hinh       1.00      1.00      1.00       111
 __label__Mang_internet_va_vien_th

In [ ]:
print('Ground\tPred\tText')
for i in range(10):
    print(str(y_test[i]) + '\t' + str(y_pred[i]) + '\t' + X_test[i])

## Save model

In [9]:
savefile = "/content/drive/MyDrive/Colab Notebooks/data mining/save/" + 'BERT_158len.bin'

In [10]:
# save model to disk
model.save(savefile)

#**Testmodel**

In [ ]:
# load model
model = load_model(savefile)

Loading model from /content/drive/MyDrive/Colab Notebooks/data mining/save/BERT_158len.bin...
Defaulting to linear classifier/regressor
Building sklearn text classifier...


In [11]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder().fit(df.true_label)

In [12]:
df_test = pd.read_excel(TEST_PATH)
df_test.tail()

,text,len
10012,tổng_hợp những món ngon khu_vực đào tấn ba_đìn...,173
10013,bản tin tài_chính kinh_doanh tối thứ_sáu tin t...,122
10014,ngang_nhiên vừa hack vừa game thủ bị ban ngay ...,32
10015,tour nước_ngoài dịp giáng_sinh năm mới giá tốt...,135
10016,phòng_không khép_kín ở ngõ xuân thuỷ giá đóng ...,59


In [13]:
xtest = pd.read_excel(TEST_PATH).text
ypred = model.predict(xtest)
true_labels = encoder.inverse_transform(ypred)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/1253 [00:00<?, ?it/s]

In [14]:
with open("/content/drive/MyDrive/Colab Notebooks/data mining/predicttion/pred_bert_158.txt", "wt") as f:
    for i in true_labels:
        f.write(i+"\n")

In [15]:
true_labels

array(['__label__Nha_dat', '__label__Mang_internet_va_vien_thong',
       '__label__Nha_dat', ..., '__label__Giai_tri', '__label__Du_lich',
       '__label__Nha_dat'], dtype=object)